In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

plt.rc('figure',figsize=(17,13))
import plotly.express as px
import plotly.graph_objs as go
import plotly.offline as pyo
from plotly.subplots import make_subplots
pyo.init_notebook_mode()


import re
import string 

import nltk
from nltk.probability import FreqDist
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk import pos_tag
from nltk.tokenize import word_tokenize

from wordcloud import WordCloud
from tqdm.auto import tqdm

In [ ]:
df=pd.read_csv('tweets.csv')
df.head()

In [ ]:
print('Rows              :',df.shape[0])
print('Columns           :',df.shape[1])
print('\nFeatures        :\n',df.columns.tolist())
print('\nMissing values  :',df.isna().sum().values.sum())
print('\nUnique values   :',df.nunique())

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer 
analyser=SentimentIntensityAnalyzer()
scores=[]
for i in range(len(df['text'])):
    score=analyser.polarity_scores(df['text'][i])
    score=score['compound']
    scores.append(score)
sentiment=[]
for i in scores:
    if i>=0.05:
        sentiment.append('Positive')
    elif i<=(-0.05):
        sentiment.append('Negative')
    else:
        sentiment.append('Neutral')
df['sentiment']=pd.Series(np.array(sentiment))

In [ ]:
df.head(3)

# Exploratory Data Analysis (EDA) 

In [ ]:
# Sentiment count
temp=df.groupby('sentiment')['text'].count().reset_index().sort_values(by='sentiment',ascending=False)
temp.style.background_gradient(cmap='Blues')

In [ ]:
plt.figure(figsize=(12,6))
sns.countplot(df['sentiment'])
plt.title('sentiment Distribution',fontsize=20)
plt.show()

In [ ]:
# Account Verified / Not verified

plt.figure(figsize=(12,6))
sns.countplot(df['user_verified'])
plt.title('Account Distribution',fontsize=20)
plt.show()

### Distribution of the favoutires recieved based on tweets sentiment and whether the account is verified/Not

In [ ]:
plt.figure(figsize=(12,6))
sns.barplot(x='user_verified',y='user_favourites',hue='sentiment',data=df)
plt.show()

### The Top 30 Most Hashtags in the tweet text data

In [ ]:
df['hashtags']=df['hashtags'].fillna('[]')

all_hashtags=[]
for i in range(len(df['hashtags'])):
    a=df['hashtags'][i].strip('][').split(',')
    for i in a:
        all_hashtags.append(i)
all_hashtags=['No hashtags' if x=='' else x for x in all_hashtags]

all_hashtags=pd.Series(np.array(all_hashtags))
print('There are {} instances of tweets in which No hashtags were used'.format(all_hashtags.value_counts()[1]))

common_hashtags=all_hashtags.value_counts().drop(labels='No hashtags')[:30].rename_axis('Common Hashtags').reset_index(name='count')
fig=px.treemap(common_hashtags,path=['Common Hashtags'],values='count',title='Top 30 Common Hashtags')
fig.show()

### Sentiment wise analysis of hashtags and words

In [ ]:
positive_tweet=df[df['sentiment']=='Positive'].reset_index()
negative_tweet=df[df['sentiment']=='Negative'].reset_index()
neutral_tweet=df[df['sentiment']=='Neutral'].reset_index()

In [ ]:
# POSITIVE TWEETS
all_positive_words=[]
all_positive_hashtags=[]
for i in range(len(positive_tweet['text'])):
    a=positive_tweet['text'][i]
    b=positive_tweet['hashtags'][i].strip('][').split(', ')
    for i in a:
        all_positive_words.append(i)
    for i in b:
        all_positive_hashtags.append(i)

all_positive_words=pd.Series(np.array(all_positive_words))
all_positive_hashtags=pd.Series(np.array(all_positive_hashtags))

In [ ]:
all_negative_words=[]
all_negative_hashtags=[]
for i in range(len(negative_tweet['text'])):
    a=negative_tweet['text'][i]
    b=negative_tweet['hashtags'][i].strip('][').split(', ')
    for i in a:
        all_negative_words.append(i)
    for i in b:
        all_negative_hashtags.append(i)
all_negative_words=pd.Series(np.array(all_negative_words))
all_negative_hashtags=pd.Series(np.array(all_negative_hashtags))

In [ ]:
all_neutral_words=[]
all_neutral_hashtags=[]
for i in range(len(neutral_tweet['text'])):
    a=neutral_tweet['text'][i]
    b=neutral_tweet['hashtags'][i].strip('][').split(', ')
    for i in a:
        all_neutral_words.append(i)
    for i in b:
        all_neutral_hashtags.append(i)
all_neutral_words=pd.Series(np.array(all_neutral_words))
all_neutral_hashtags=pd.Series(np.array(all_neutral_hashtags))

In [ ]:
common=set(all_positive_words).intersection(set(all_negative_words)).intersection(set(all_neutral_words))
common_list=list(common)

common_words=all_negative_words.value_counts().drop(labels=common_list)[:30].rename_axis('common negative words').reset_index(name='count')
fig=px.treemap(common_words,path=['common negative words'],values='count',title='Top 30 Unique words in negative Tweets')
fig.show()

common_words=all_positive_words.value_counts().drop(labels=common_list)[:30].rename_axis('common positive words').reset_index(name='count')
fig=px.treemap(common_words,path=['common positive words'],values='count',title='TOp 30 Unique words in Positive Tweets')
fig.show()

common_words=all_neutral_words.value_counts().drop(labels=common_list)[:30].rename_axis('common neutral words').reset_index(name='count')
fig=px.treemap(common_words,path=['common neutral words'],values='count',title='Top 30 Unique words in neutral Tweets')
fig.show()

### Source Distribution of Tweets

In [ ]:
data_=df['source'].value_counts().reset_index()

trace1=go.Bar(x=['Twitter Web App', 'Twitter for iPhone', 'Twitter for Android','LinkedIn', 'HubSpot', 'IFTTT', 
                 'Hypefury', 'Sprout Social','Revive Social App', 'Buffer'],
              y=data_['source'],marker=dict(color='rgb(250,13,92)',
              line=dict(color='rgb(0,0,0)',width=1.5)),text=data_['source'],textposition='outside')

layout=go.Layout(template='plotly_dark',title='Top 10 Most Source Disrtibution Of Tweets',xaxis=dict(title='Source'),
                 yaxis=dict(title='Count'),height=700)
fig=go.Figure(data=[trace1],layout=layout)
fig.show()

### Here new feature "sentiment" is created. corresponding to each tweet text.

### Here new feature "sentiment" is created. corresponding to each tweet text.

### Verified Account of Tweet

In [ ]:
data_verified=df[df['user_verified']==True].reset_index()
data_not_verified=df[df['user_verified']==False].reset_index()

In [ ]:
data_=data_verified['source'].value_counts().reset_index()

trace1=go.Bar(x=['Twitter Web App', 'Twitter for iPhone', 'Twitter for Android','LinkedIn', 'HubSpot', 'IFTTT', 
                 'Hypefury', 'Sprout Social','Revive Social App', 'Buffer',],y=data_['source'],
            marker=dict(color='rgb(250,13,92)',line=dict(color='rgb(0,0,0)',width=1.5)),text=data_['source'],
             textposition='outside')
layout=go.Layout(template='plotly_dark',title='Top 20 Most Source Distribution of Twwets From Verified Accounts',xaxis=dict(title='Source'),
                yaxis=dict(title='Count'),height=650)
fig=go.Figure(data=[trace1],layout=layout)
fig.show()

In [ ]:
all_hashtags=[]
for i in range(len(data_verified['hashtags'])):
    a=data_verified['hashtags'][i].strip('][').split(', ')
    for i in a:
        all_hashtags.append(i)
all_hashtags=pd.Series(np.array(all_hashtags))
common_hashtags=all_hashtags.value_counts()[:30].rename_axis('common hashtags').reset_index(name='count')
fig=px.treemap(common_hashtags,path=['common hashtags'],values='count',title='30 Most common hashtags by Verified Accounts')
fig.show()

### Tweets Sentiment Analysis Based on Location

In [ ]:
df['user_location']=df['user_location'].fillna('')

postive_tweet=df[df['sentiment']=='Positive'].reset_index()
negative_tweet=df[df['sentiment']=='Negative'].reset_index()
neutral_tweet=df[df['sentiment']=='Neutral'].reset_index()

pos_location=positive_tweet['user_location']
neg_location=negative_tweet['user_location']
neu_location=neutral_tweet['user_location']

common=set(pos_location).intersection(set(neg_location)).intersection(set(neu_location))
common_list=list(common)

common_words=neg_location.value_counts().drop(labels=common_list)[:10].rename_axis('common negative location').reset_index(name='count')
fig=px.treemap(common_words,path=['common negative location'],values='count',title='10 Top Unique negative Tweets Location')
fig.show()

common_words=pos_location.value_counts().drop(labels=common_list)[:10].rename_axis('common positive location').reset_index(name='count')
fig=px.treemap(common_words,path=['common positive location'],values='count',title='10 Top Unique positive Location')
fig.show()

common_words=neu_location.value_counts().drop(labels=common_list)[:10].rename_axis('common neutral location').reset_index(name='count')
fig=px.treemap(common_words,path=['common neutral location'],values='count',title='10 top Unique Neutral Location')
fig.show()

### Accounts with highest followers analysis based on the tweet sentiment

In [ ]:
fig,(ax1,ax2,ax3)=plt.subplots(3,1,figsize=(10,16))
sns.barplot(x='user_followers',y='user_name',orient='h',ax=ax1,palette=['b'],data=df[df['sentiment']=='Positive']\
            .drop_duplicates(subset=['user_name'])\
            .sort_values(by=['user_followers'],ascending=False)[:10])
ax1.set_title('Top 10 Accounts with Highest Followers whos tweets are Positive',fontsize=19,fontweight='bold')

sns.barplot(x='user_followers',y='user_name',orient='h',ax=ax2,palette=['g'],data=df[df['sentiment']=='Negative']\
           .drop_duplicates(subset='user_name')\
           .sort_values(by='user_followers',ascending=False)[:10])
ax2.set_title('Top 10 Accounts with highest Follwers whos tweets are Negative',fontsize=19,fontweight='bold')

sns.barplot(x='user_followers',y='user_name',orient='h',ax=ax3,palette=['r'],data=df[df['sentiment']=='Neutral']\
           .drop_duplicates(subset='user_name')\
           .sort_values(by='user_followers',ascending=False)[:10])
ax3.set_title('Top 10 Accounts with highest follwers whos tweets are neutral',fontsize=18,fontweight='bold')

### User with more friends

In [ ]:
df[df['user_friends'] == df['user_friends'].max()]['user_name'].iloc[0]

### User followers in Thounds

In [ ]:
most_pop=df.sort_values('user_followers',ascending=False)[['user_name','user_followers']].head(12)

most_pop['user_followers1']=most_pop['user_followers']/1000


plt.figure(figsize=(12,8))

sns.barplot(data=most_pop, y='user_name',x='user_followers1',color='c')
plt.xticks(fontsize=8,rotation=0)
plt.yticks(fontsize=15,rotation=0)
plt.xlabel('User followers in Thounds',fontsize=20)
plt.ylabel('')
plt.title('Followers',fontsize=15)

# Cleaning 

In [ ]:

def remove_line_breaks(text):
    text = text.replace('\r', ' ').replace('\n', ' ')
    return text

#remove punctuation
def remove_punctuation(text):
    re_replacements = re.compile("__[A-Z]+__")  # such as __NAME__, __LINK__
    re_punctuation = re.compile("[%s]" % re.escape(string.punctuation))
    '''Escape all the characters in pattern except ASCII letters and numbers'''
    tokens = word_tokenize(text)
    tokens_zero_punctuation = []
    for token in tokens:
        if not re_replacements.match(token):
            token = re_punctuation.sub(" ", token)
        tokens_zero_punctuation.append(token)
    return ' '.join(tokens_zero_punctuation)


def remove_special_characters(text):
    text = re.sub('[^a-zA-z0-9\s]', '', text)
    return text

def lowercase(text):
    text_low = [token.lower() for token in word_tokenize(text)]
    return ' '.join(text_low)

def remove_stopwords(text):
    stop = set(stopwords.words('english'))
    word_tokens = nltk.word_tokenize(text)
    text = " ".join([word for word in word_tokens if word not in stop])
    return text

def remove_one_character_words(text):
    '''Remove words from dataset that contain only 1 character'''
    text_high_use = [token for token in word_tokenize(text) if len(token)>1]      
    return ' '.join(text_high_use)   
    
#%%
# Stemming with 'Snowball stemmer" package
def stem(text):
    stemmer = nltk.stem.snowball.SnowballStemmer('english')
    text_stemmed = [stemmer.stem(token) for token in word_tokenize(text)]        
    return ' '.join(text_stemmed)

def lemma(text):
    wordnet_lemmatizer = WordNetLemmatizer()
    word_tokens = nltk.word_tokenize(text)
    text_lemma = " ".join([wordnet_lemmatizer.lemmatize(word) for word in word_tokens])       
    return ' '.join(text_lemma)

def sentence_word(text):
    word_tokens = nltk.word_tokenize(text)
    return word_tokens
#break paragraphs to sentence token 
def paragraph_sentence(text):
    sent_token = nltk.sent_tokenize(text)
    return sent_token    

def tokenize(text):
    """Return a list of words in a text."""
    return re.findall(r'\w+', text)

def remove_numbers(text):
    no_nums = re.sub(r'\d+', '', text)
    return ''.join(no_nums)

def clean_text(text):
    _steps = [
    remove_line_breaks,
    remove_one_character_words,
    remove_special_characters,
    lowercase,
    remove_punctuation,
    remove_stopwords,
    stem,
    remove_numbers
]
    for step in _steps:
        text=step(text)
    return text   

In [ ]:
df['text']=df['text'].astype(str)
df['text']=[x.replace(':',' ') for x in df['text']]

In [ ]:
df['clean_text']=pd.Series([clean_text(i) for i in tqdm(df['text'])])

In [ ]:
df.loc[:,['text','clean_text']]

## Most discussed terms

In [ ]:
words=df['clean_text'].values

In [ ]:
ls=[]
for i in words:
    ls.append(str(i))

In [ ]:
plt.figure(figsize=(16,13))
wc=WordCloud(background_color='lightblue',colormap='Set2',max_words=1000,max_font_size=200,width=1600,height=800)
wc.generate(" ".join(ls))
plt.title('Most discussed terms',fontsize=20)
plt.imshow(wc.recolor(colormap='Set2',random_state=17),alpha=0.98,interpolation='bilinear')
plt.axis('off')